In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, ReLU
from keras.metrics import MeanSquaredError
from sklearn.metrics import mean_squared_error, r2_score
from keras.optimizers import Adam
from keras import backend as K

# 1차 LSTM 모델 생성

In [76]:
data = pd.read_csv('C:/Users/LG/Desktop/Junior/1학기/통계데이터 분석 공모전/merged_predict.csv')

features = data[['검색건수', '관외 이동량', '숙박업소 개수']]
target = data['방문자수'].values

# 데이터 정규화
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()
scaled_features = feature_scaler.fit_transform(features)
scaled_target = target_scaler.fit_transform(target.reshape(-1, 1))

In [77]:
# 시퀀스 데이터 변환 함수
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 3
X, y = create_dataset(scaled_features, scaled_target, time_steps)

#데이터셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [78]:
# 사용자 정의 손실 함수 정의
def custom_loss(y_true, y_pred):
    penalty = 1000 * K.sum(K.relu(-y_pred))  # 음수 예측에 대한 큰 패널티 추가
    return K.mean(K.square(y_pred - y_true)) + penalty

# LSTM 모델 생성
model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(50),
    Dense(1)
])

# 모델 컴파일
model.compile(loss='mean_squared_error', optimizer='adam')

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=2)

Epoch 1/100
49/49 - 2s - 47ms/step - loss: 0.0294
Epoch 2/100
49/49 - 0s - 4ms/step - loss: 0.0199
Epoch 3/100
49/49 - 0s - 4ms/step - loss: 0.0195
Epoch 4/100
49/49 - 0s - 4ms/step - loss: 0.0192
Epoch 5/100
49/49 - 0s - 4ms/step - loss: 0.0191
Epoch 6/100
49/49 - 0s - 5ms/step - loss: 0.0193
Epoch 7/100
49/49 - 0s - 4ms/step - loss: 0.0190
Epoch 8/100
49/49 - 0s - 4ms/step - loss: 0.0190
Epoch 9/100
49/49 - 0s - 3ms/step - loss: 0.0187
Epoch 10/100
49/49 - 0s - 3ms/step - loss: 0.0188
Epoch 11/100
49/49 - 0s - 3ms/step - loss: 0.0187
Epoch 12/100
49/49 - 0s - 4ms/step - loss: 0.0188
Epoch 13/100
49/49 - 0s - 4ms/step - loss: 0.0186
Epoch 14/100
49/49 - 0s - 5ms/step - loss: 0.0185
Epoch 15/100
49/49 - 0s - 4ms/step - loss: 0.0185
Epoch 16/100
49/49 - 0s - 4ms/step - loss: 0.0185
Epoch 17/100
49/49 - 0s - 5ms/step - loss: 0.0185
Epoch 18/100
49/49 - 0s - 5ms/step - loss: 0.0186
Epoch 19/100
49/49 - 0s - 4ms/step - loss: 0.0183
Epoch 20/100
49/49 - 0s - 4ms/step - loss: 0.0184
Epoch 21

In [79]:
# 예측 및 성능 평가
y_pred = model.predict(X_test)
mse = MeanSquaredError()
mse_value = mse(y_test, y_pred).numpy()
rmse = mse_value ** 0.5
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse_value}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared: {r2}')

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Mean Squared Error: 0.014458870515227318
Root Mean Squared Error: 0.12024504362021463
R-squared: 0.4791068407054947


In [80]:
# 2024년 1월~5월 예측
predict_data = data[data['일시'].isin([1, 2, 3, 4, 5])]
predict_features = predict_data[['검색건수', '관외 이동량', '숙박업소 개수']]
scaled_predict_features = feature_scaler.transform(predict_features)

# 시퀀스 데이터 생성
X_predict, _ = create_dataset(scaled_predict_features, np.zeros(len(predict_features)), time_steps)

# 예측 수행
predicted_visitors = model.predict(X_predict)

# NaN 값으로 채워진 배열을 생성
nan_array = np.empty((time_steps, 1))
nan_array[:] = np.nan

# 예측 결과 앞에 NaN을 추가
adjusted_predictions = np.vstack([nan_array, predicted_visitors])

# 예측 결과를 원래 스케일로 변환 후 정수로 변환
adjusted_predictions = target_scaler.inverse_transform(adjusted_predictions)
adjusted_predictions = np.nan_to_num(adjusted_predictions, nan=0)
adjusted_predictions = np.clip(adjusted_predictions, 0, None).astype(int)

# 결과를 DataFrame에 추가
predict_data = predict_data.assign(**{'예측 방문자수': adjusted_predictions.ravel()})

# 결과 출력
print(predict_data)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
      일시          지역명     방문자수    검색건수  관외 이동량  숙박업소 개수  예측 방문자수
0      1  강원특별자치도 춘천시  8979957  229813  444582      312        0
1      1  강원특별자치도 원주시  9530785  199254  482623      133        0
2      1  강원특별자치도 강릉시  8031760  452294  565563      658        0
3      1  강원특별자치도 동해시  2946561   64643  167985      101  4771051
4      1  강원특별자치도 태백시  1491579   32726   92434       20  3649397
...   ..          ...      ...     ...     ...      ...      ...
1859   5     충청북도 증평군  1388821   29203   92801        6  1600492
1860   5     충청북도 진천군  2551282   84387  260404       39  1495953
1861   5     충청북도 괴산군  1536345   29203  158593       74  2388364
1862   5     충청북도 음성군  3975691   78947  317368       29  2401446
1863   5     충청북도 단양군  1440243  101057  150693      189  2589977

[815 rows x 7 columns]


In [81]:
zero_predictions = predict_data[predict_data['예측 방문자수'] == 0]
print(zero_predictions)

#왜 0으로 예측하지........

   일시          지역명     방문자수    검색건수  관외 이동량  숙박업소 개수  예측 방문자수
0   1  강원특별자치도 춘천시  8979957  229813  444582      312        0
1   1  강원특별자치도 원주시  9530785  199254  482623      133        0
2   1  강원특별자치도 강릉시  8031760  452294  565563      658        0


# 실제 2024년 1~5월 방문자수와 비교

In [82]:
real = pd.read_csv('C:/Users/LG/Desktop/Junior/1학기/통계데이터 분석 공모전/real_predict.csv')
print(real)

      일시       방문자 수          지역명
0      1   1452663.0  강원특별자치도 고성군
1      1   9024519.0  강원특별자치도 춘천시
2      1   9956163.0  강원특별자치도 원주시
3      1   8218107.0  강원특별자치도 강릉시
4      1   2874821.0  강원특별자치도 동해시
...   ..         ...          ...
1260   5  15900000.0     충청북도 증평군
1261   5  29900000.0     충청북도 진천군
1262   5  17900000.0     충청북도 괴산군
1263   5  40500000.0     충청북도 음성군
1264   5  16000000.0     충청북도 단양군

[1265 rows x 3 columns]


In [83]:
merged_df = pd.merge(real, predict_data, on=['일시', '지역명'])
merged_df=merged_df[['일시','지역명','방문자 수','예측 방문자수']]
merged_df

,일시,지역명,방문자 수,예측 방문자수
0,1,강원특별자치도 고성군,1452663.0,1848361
1,1,강원특별자치도 춘천시,9024519.0,0
2,1,강원특별자치도 원주시,9956163.0,0
3,1,강원특별자치도 강릉시,8218107.0,0
4,1,강원특별자치도 동해시,2874821.0,4771051
...,...,...,...,...
810,5,충청북도 증평군,15900000.0,1600492
811,5,충청북도 진천군,29900000.0,1495953
812,5,충청북도 괴산군,17900000.0,2388364
813,5,충청북도 음성군,40500000.0,2401446


In [84]:
# 방문자 수와 예측 방문자 수의 차이 계산
merged_df['오차'] = merged_df['방문자 수'] - merged_df['예측 방문자수']

# MSE 계산
mse = mean_squared_error(merged_df['방문자 수'], merged_df['예측 방문자수'])

print("Mean Squared Error (MSE): {:.0f}".format(mse))

Mean Squared Error (MSE): 717809431446696


In [85]:
merged_df

,일시,지역명,방문자 수,예측 방문자수,오차
0,1,강원특별자치도 고성군,1452663.0,1848361,-395698.0
1,1,강원특별자치도 춘천시,9024519.0,0,9024519.0
2,1,강원특별자치도 원주시,9956163.0,0,9956163.0
3,1,강원특별자치도 강릉시,8218107.0,0,8218107.0
4,1,강원특별자치도 동해시,2874821.0,4771051,-1896230.0
...,...,...,...,...,...
810,5,충청북도 증평군,15900000.0,1600492,14299508.0
811,5,충청북도 진천군,29900000.0,1495953,28404047.0
812,5,충청북도 괴산군,17900000.0,2388364,15511636.0
813,5,충청북도 음성군,40500000.0,2401446,38098554.0
